# 180801_MK_Analysis

In [1]:
%matplotlib inline
import glob
import h5py
import itertools
import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import os.path
import pandas as pd
import re
from shutil import copy2, move
import skimage
import skimage.exposure
import skimage.io
import subprocess
from tqdm._tqdm_notebook import tqdm_notebook

C:\Users\Prakrith\AppData\Local\conda\conda\envs\bioimg\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Alphanumeric sorter --> keep everything organized
def sorted_nicely(l):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key = alphanum_key)

In [3]:
# Project 

image_directory = r"C:\Users\Prakrith\Dropbox (Partners HealthCare)\WORK\SHARED\ARW\180717_SP"

In [4]:
# Application paths for subprocess

path_to_ilastik = r"C:\Program Files\ilastik-1.3.0b4\run-ilastik.bat"

path_to_cellprofiler = r"C:\Program Files (x86)\CellProfiler\CellProfiler.exe"

1) **CPA Rules** - Cell Profiler Analyst Rules 
2) **project** means ilastik project file (ilp) 
3) **cp_pipeline** refers to CellProfiler pipeline

In [5]:
# Human pipeline variables

CPA_Rules_human = r'C:\Users\Prakrith\Desktop\CPTemp_in\fgb_rules_human.txt'

path_to_project_human = r"C:\Users\Prakrith\Documents\GitHub\Test\ilps\180805_HumanMK.ilp"

path_to_cp_pipeline_human = r"C:\Users\Prakrith\Documents\GitHub\Test\pipelines\180527_HP.cppipe"

In [6]:
# Murine pipeline variables

CPA_Rules_mice = r'C:\Users\Prakrith\Desktop\CPTemp_in\fgb_rules_pplt.txt'

path_to_project_mice = r"C:\Users\Prakrith\Documents\GitHub\Test\ilps\180805_FLMK.ilp"

path_to_cp_pipeline_mice2 = r"C:\Users\Prakrith\Documents\GitHub\Test\pipelines\180804_MPFluo.cppipe"

path_to_cp_pipeline_mice1 = r"C:\Users\Prakrith\Documents\GitHub\Test\pipelines\180804_MP.cppipe"

path_to_skel_pipeline_mice = r"C:\Users\Prakrith\Documents\GitHub\Test\pipelines\Kyle_Skel.cppipe"

In [7]:
# Fluorescence pipeline variables

path_to_coloc_pipeline =  r'C:\Users\Prakrith\Documents\GitHub\Test\pipelines\180712_TMK.cppipe'

path_to_rfp_pipeline = r'C:\Users\Prakrith\Documents\GitHub\Test\pipelines\180728_RFP.cppipe'

path_to_gfp_pipeline = r'C:\Users\Prakrith\Documents\GitHub\Test\pipelines\180728_GFP.cppipe'

In [ ]:
# image_directory = r"/home/prakrith/Applications/test2/"
# path_to_ilastik = r"/home/prakrith/Applications/ilastik-1.3.0-Linux/run_ilastik.sh"
# path_to_project = r"/home/prakrith/Applications/ilastik-1.3.0-Linux/180517_Zoom.ilp"

# Unpack (or separate) input images into single files.

To effectively use ilastik some formating must be done before and after ilastik processes the images. The input is assumed to be a time-series of images stored in a multi-page TIFF.

## Update input image variables
Update the variable *image_directory* with the path to a folder that contains the input images. Update the *regex_image* variable to process only the images that match the regular expression. If the *regex_image* variable is equal to `(.*)\.tif`, then any *.tif* in the folder will be processed.

If a filename matches the *regex_single* regular expression, then it is assumed that this image has already been unpacked. An unpacked single image will have timepoint appended to the end of the file following the pattern `\-\d{4}\.tif`

*path_to_ilastik* is a string with the path to the ilastik software for [running headless](http://ilastik.org/documentation/basics/headless.html).

In [8]:
regex_image = "(.*)\.tif" #stack
regex_single = ".*\-\d{4}\.tif" #slice
re_image = re.compile(regex_image)
re_single = re.compile(regex_single)

In [9]:
os.makedirs(os.path.join(image_directory, "single_images"), exist_ok=True)
imdir_single = os.path.join(image_directory,"single_images")
os.makedirs(os.path.join(image_directory, "ilastik"), exist_ok=True)
imdir_ilastik = os.path.join(image_directory,"ilastik")
os.makedirs(os.path.join(image_directory, "output"), exist_ok=True)
output_directory = os.path.join(image_directory,"output")

In [ ]:
# image_directory = r"C:\Users\Prakrith\Desktop\test"
# imdir_single = r"C:\Users\Prakrith\Desktop\test\single_images"
# imdir_ilastik = r"C:\Users\Prakrith\Desktop\test\ilastik"
# output_directory = r"C:\Users\Prakrith\Desktop\test\output"

# Methods to import image metadata
* *is_my_file* will use the regular expression to filter image files to be processed.
* *make_dict* parses a file to be processed and places metadata into a dictionary.

Parse the files to be processed and then place the metadata into a Pandas dataframe.

In [10]:
def is_my_file(filename, re_image, re_single):
    
    mybool = False
    
    if (    re_image.match(filename) != None 
        and re_single.match(filename) == None
       ):
        
        mybool = True
        
    return mybool


def make_dict(filename, path, re_obj):
    
    my_dict = re_obj.match(filename).groupdict()
    
    my_dict["filename"] = filename
    
    my_dict["path"] = path
    
    return my_dict

## Unpack the multi-page phase TIFF images
For every image, create a single image file for each timepoint. The input images are assumed to be RGB, which has 3 dimensions (length, width, color). The multipage TIFF of RGB images will have 4 dimensions (timepoints, length, width, color). 

*If the upstream workflow changes and the input image format is altered, then the conditional logic below will need to be updated, specifically the logic based on the shape of the input images.*

### Are the images across experiments similar enough to treat equally
One concern is that overfitting from training a classification model either through ilastik or CellProfiler analyst. The training set needs to be representative of the possibility space. This is accomplished by choosing a large enough image set that includes images of all states of interest including undifferentiated and fully differentiated megakaryocytes.

We also want to eliminate noise from known sources of variablity that could potentially weaken the classifier. The primary sources of noise in the images will be non-uniform illumination and differences in exposure. Non-uniform illumination is difficult to correct, because the background is actually in the middle of the intensity range and the signal occupies both high and low intensities.



In [ ]:
def df_stack_image(image_df,folder):
    
    for n in image_df.index:
        
        im = skimage.io.imread(os.path.join(image_df["path"][n], image_df["filename"][n]))
        
        if len(im.shape) < 4:
            
            retest = re_image.match(image_df["filename"][n])
            retest.group(1)
            fname = "{0}-{1:04d}.tif".format(retest.group(1), 0)
            im2 = skimage.color.rgb2gray(im)
            im2 = skimage.img_as_ubyte(im2)
            skimage.io.imsave(os.path.join(image_df["path"][n], fname), im2)
            
        else:
            
            number_of_timepoints = im.shape[0]
            
            for i in range(number_of_timepoints):
                
                retest = re_image.match(image_df["filename"][n])
                retest.group(1)
                fname = "{0}-{1:04d}.tif".format(retest.group(1), i)
                im2 = skimage.color.rgb2gray(im[i,:,:,:])
                im2 = skimage.img_as_ubyte(im2)
                skimage.io.imsave(os.path.join(image_df["path"][n], folder, fname), im2)

In [ ]:
image_files_dict = [make_dict(f, image_directory, re_image) for f in os.listdir(image_directory) if is_my_file(f, re_image, re_single)]
image_df = pd.DataFrame(image_files_dict)

In [ ]:
if image_df.empty is False:
    
    df_stack_image(image_df,"single_images")
    
else:
    
    print("no phase images to unpack.")

# Fluorescence Check

Check to see if fluorescence folders (**rfp** or **gfp**) containing fluo stacks exist in the main image directory - if yes Proplatelet Production pipeline will generate MK label images along with the usual proplatelet labels to quantify fluo within both populations. If no fluo folders are detected, only the proplatelet labels will be created for the skel pipe, reducing processing time & project size.

In [ ]:
if os.path.isdir(image_directory + '\\' + 'rfp') == True:

    for dirpath, dirnames, files in os.walk(image_directory + '\\' + 'rfp'):
        if files:    
            rfp = os.path.join(image_directory,"rfp")
            os.makedirs(os.path.join(rfp, "rfp_single"), exist_ok=True)
            rfp_single = os.path.join(rfp, "rfp_single")
            rfp_files_dict = [make_dict(f, rfp, re_image) for f in os.listdir(rfp) if is_my_file(f, re_image, re_single)]
            rfp_df = pd.DataFrame(rfp_files_dict)
            
            if rfp_df.empty is False:
                df_stack_image(rfp_df,"rfp_single")
                rfp_list = sorted_nicely(glob.glob(os.path.join(rfp_single,"*.tif")))
                
                if len(rfp_list) != len(single_list):
                    print("number of rfp single_images doesn't match number of phase single_images")
                    break;
                else:
                    pass;
            else:
                break;
#                 rfp_list = sorted_nicely(glob.glob(os.path.join(gfp_single,"*.tif")))
#                 print("no rfp images to unpack.")
                
        if not files:
            print(dirpath, 'is empty')
            
else:
    rfp = None;

if os.path.isdir(image_directory + '\\' + 'gfp') == True:
    
    for dirpath, dirnames, files in os.walk(image_directory + '\\' + 'gfp'):
        if files:
            
            gfp = os.path.join(image_directory,"gfp")
            os.makedirs(os.path.join(gfp, "gfp_single"), exist_ok=True)
            gfp_single = os.path.join(gfp, "gfp_single")
            gfp_files_dict = [make_dict(f, gfp, re_image) for f in os.listdir(gfp) if is_my_file(f, re_image, re_single)]
            gfp_df = pd.DataFrame(gfp_files_dict)
                    
            if gfp_df.empty is False:
                
                df_stack_image(gfp_df,"gfp_single")
                gfp_list = sorted_nicely(glob.glob(os.path.join(gfp_single,"*.tif")))
                
                if len(gfp_list) != len(single_list):
                    print("number of gfp single_images doesn't match number of phase single_images")
                    break;
                else:
                    pass;          
            else:
                break;
#                 gfp_list = sorted_nicely(glob.glob(os.path.join(gfp_single,"*.tif")))
#                 print("no gfp images to unpack.")
                
        if not files:
            print(dirpath, 'is empty')
    
else:
    gfp = None;

In [ ]:
if rfp is not None and gfp is not None:
    fluo = 1 #both
elif rfp is not None and gfp is None:
    fluo = 2 #rfp
elif rfp is None and gfp is not None:
    fluo = 3 #gfp
else:
    fluo = False

# User Query

In [14]:
ans1 = ['1']
ans2 = ['2']

celltype = str(input("[1]Human MK or [2]Mice MK ?"))

if celltype in ans1:
    
    #directory with location of human MK pplt CellProfiler Analyst Rules
    
    CPA_Rules = CPA_Rules_human 
    
    #copy rules.txt to project folder
    copy2(CPA_Rules, image_directory)
    
    path_to_project = path_to_project_human
    path_to_cp_pipeline = path_to_cp_pipeline_human
    
elif celltype in ans2:
        
    CPA_Rules = CPA_Rules_mice
    copy2(CPA_Rules, image_directory)
    
    path_to_project = path_to_project_mice
        
    if fluo is not False:
        # Alternative pipe for generating MK labels for fluo analysis
        path_to_cp_pipeline = path_to_cp_pipeline_mice2
    else:
        # Standard pplt production pipeline
        path_to_cp_pipeline = path_to_cp_pipeline_mice1
    
else:
    print("Invalid input")

[1]Human MK or [2]Mice MK ?2


In [15]:
# Include the number of scans to account for mean/std_dev when generating graphs downstream
    
ans3 = ['1','2','3','4','9']

numscans = input("Number of scans per well?")

if numscans in ans3:
    numscans = int(numscans)
else:
    print("Invalid number of scans")

Number of scans per well?4


# ilastikProcessing

Using the single images created earlier, process the images using ilastik. First, create another dataframe with the single image metadata. Note, this has been written for running on Windows.

## Process ilastik output for CellProfiler
ilastik will output an HDF5 file that must be parsed for use as input to CellProfiler. This workflow assumes the default export settings are being used in ilastik. We have observed performance costs when changing the exporting settings to formats beyond the standard ilastik HDF5 file. For example, exporting TIFF images changes the shape of the exported data from yxc (the default) to cyx. This rearrangement will cause downstream errors, because the code as written expects the channel to be the third dimension.

### ilastik stage-2 labels
The ilastik project file(s) (*.ilp*) has the following labels that are stored in the same order within the HDF5 output.
1. background
1. cell_boundary
1. cell
1. protrusion

In [ ]:
def is_my_file2(filename, re_obj):
    
    mybool = False
    
    if re_obj.match(filename) != None:
        
        mybool = True
        
    return mybool

In [ ]:
def df_ilastik(p):
    
    filename = os.path.join(p["path"], p["filename"])
    
    filename_noext = os.path.splitext(p["filename"])[0]
    
    filename_h5 = "{}_Probabilities Stage 2.h5".format(filename_noext)
    
    # Run ilastik using subprocess
    
    command = (path_to_ilastik,"--headless","--export_source=probabilities stage 2","--output_format=hdf5",
               r"--project={}".format(path_to_project),filename)
    
    process = subprocess.Popen(command, stdout=subprocess.PIPE)
    
    out, err = process.communicate()
    
    # unpack the HDF5 file
    
    label_list = ["background", "protrusion", "cell_boundary", "cell"]
    
    path_h5 = os.path.join(p["path"], filename_h5)
    
    with h5py.File(path_h5, "r") as ilastik_hdf5:
    
        ilastik_probabilities = ilastik_hdf5["exported_data"].value
    
        for i in range(ilastik_probabilities.shape[2]):
            im = skimage.img_as_uint(ilastik_probabilities[:, :, i])
        
            filename_slice = "{}_{}_prbstg2_{}.png".format(filename_noext, label_list[i], i)
        
            skimage.io.imsave(os.path.join(p["path"], "..", "ilastik", filename_slice), im)
    
    os.remove(path_h5)

In [ ]:
image_files_dict = [make_dict(f, imdir_single, re_single) for f in os.listdir(imdir_single) if is_my_file2(f, re_single)]
image_df = pd.DataFrame(image_files_dict)

In [ ]:
# tqdm_notebook.pandas(desc="run ilastik")
# _ = image_df.progress_apply(df_ilastik, axis=1)

In [ ]:
_ = image_df.apply(df_ilastik, axis=1)

#### After ilastikProcessing on HMS O2, issues have been noticed where probability maps are uncapitalized when exported; CP processing is CASE SENSITIVE.

In [ ]:
# for file in os.listdir(imdir_ilastik):
#     first = file.split('-', 1)[0].replace('.', '').upper()
#     last = file.split('-', 1)[1]
#     newName = str(first + '-' + last)
#     os.rename(os.path.join(imdir_ilastik, file), os.path.join(imdir_ilastik, newName))

# ppltProcessing

## Make a filelist
Add the paths to each file that will be processed by CellProfiler into a text file.

In [16]:
single_list = sorted_nicely(glob.glob(os.path.join(imdir_single,"*.tif")))
ilastik_list = sorted_nicely(glob.glob(os.path.join(imdir_ilastik,"*.png")))
big_list = single_list + ilastik_list
big_list = sorted_nicely(big_list)
with open(os.path.join(image_directory,"pplt_list.txt"), 'w') as f:
    for item in big_list:
        f.write("{}\n".format(item))

## (1) Proplatelet Production Pipeline
Use subprocess to run CellProfiler on the images to be processed.

Note, that a model that filters protrusions was trained in CellProfiler Analyst outside of this workflow. The model has to be in the input folder to be found by CellProfiler.

In [ ]:
command = (path_to_cellprofiler,"--run-headless","--pipeline={}".format(path_to_cp_pipeline),
           "--file-list={}".format(os.path.join(image_directory,"pplt_list.txt")),
           "--image-directory={}".format(image_directory),"--output-directory={}".format(output_directory))

process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    
out, err = process.communicate()

## (1) Quantify Proplatelet Production
From the csvs generated by CellProfiler, the file 'results_Image' is parsed. Proplatelet production & area (mm^2) are quantified.

In [ ]:
# move raw "results_" files to separate folder
os.makedirs(os.path.join(output_directory, "raw_files"), exist_ok=True)
dst = os.path.join(output_directory, "raw_files")

for filename in os.listdir(output_directory):
    if filename.startswith('results'):
        src = output_directory + "\\" + filename
        newdst = dst + '\\' + filename
        move(src, newdst)

In [ ]:
result = open(os.path.join(dst,r'results_Image.csv')); 
df = pd.read_csv(result,index_col = "URL_phase");
df = df.reindex(index=sorted_nicely(df.index));
df.drop(df.columns[[4,5,8,10,11,12,13,14,15,16]], axis=1, inplace=True);

In [ ]:
def ppltpct(pplt,mk):
    try:
        return ((pplt/(mk+pplt))*100) #(count_proplatelet/(count_meg+count_proplatelet)*100)
    except ZeroDivisionError:
        pass;
    
def area_mm(area_px):
    return (((area_px * 2159000) / 1447680)/1000000); #[(area * total_area_um)/total_area_px)/1x10^6] -> area mm²

In [ ]:
df['PpltPct'] = df.apply(lambda row: ppltpct(row['Count_proplatelets'], row['Count_megs']), axis=1)
df['Area_MK'] = df.apply(lambda row: area_mm(row['AreaOccupied_AreaOccupied_megs']), axis=1)
df['Area_Pplt'] = df.apply(lambda row: area_mm(row['AreaOccupied_AreaOccupied_proplatelets']), axis=1)
df['Perimeter_MK'] = df.apply(lambda row: area_mm(row['AreaOccupied_Perimeter_megs']), axis=1)
df['Perimeter_Pplt'] = df.apply(lambda row: area_mm(row['AreaOccupied_Perimeter_proplatelets']), axis=1)

In [ ]:
df.drop(df.columns[[0,1,2,3,4,5]], axis=1, inplace=True);
df = df.set_index("ImageNumber");

In [ ]:
def df_format(directory,single_list,df):
    stack_list=[]
    for filename in os.listdir(directory):
        if filename.endswith('.tif'):
            stack_list.append(filename)
        
    stack_list = sorted_nicely(stack_list)
    t = int(len(single_list)); #total num images
    n = int((len(single_list) / len(stack_list))); # slices per stack
    
    df2 = pd.DataFrame();
    for i in range(0,t,n):
        slc = df.iloc[i:i+n];
        slc = slc.reset_index(drop=True);
        df2 = pandas.concat([df2,slc],axis=1,ignore_index=True); #iter df by stack length (n), and concat 
        
    return stack_list,n,df2;

In [ ]:
# df2 returns as a tuple (stack_list,number of timepoints,df2)
df2 = df_format(image_directory,single_list,df)
phase_stack_list = df2[0]
n = df2[1]
df2 = df2[2]

In [ ]:
def csv_format(df,stack_list,n,output_dir,name): #dataframe,stack_list,num_timepoints,output_directory,csv output name
    df.columns = stack_list;
    df['Timepoint'] = list(range(1,n+1));
    df = df.set_index("Timepoint");
    df.to_csv(os.path.join(output_dir,name));

In [ ]:
csv_format(df2.loc[:,::5],phase_stack_list,n,output_directory,r'Pplt_Pct.csv');
csv_format(df2.loc[:,1::5],phase_stack_list,n,output_directory,r'Area_MK.csv');
csv_format(df2.loc[:,2::5],phase_stack_list,n,output_directory,r'Area_Pplt.csv');
csv_format(df2.loc[:,3::5],phase_stack_list,n,output_directory,r'Perimeter_MK.csv');
csv_format(df2.loc[:,4::5],phase_stack_list,n,output_directory,r'Perimeter_Pplt.csv');

## (1) Graphs

- Per well graphs depicting total integrated_intensity of both mk/pplt objects over time are generated (w/ mean & std_dev) automatically. Use the formatted csvs for more specific plotting.

In [ ]:
well_list = sorted_nicely(list(set([s.replace('', '')[:-6] for s in phase_stack_list])))
os.makedirs(os.path.join(output_directory, "graphs"), exist_ok=True)
graph_directory = os.path.join(output_directory, "graphs")

In [ ]:
def plot(df,folder,numscans,well_list,ylabel):
    mean_df = df.groupby(np.arange(len(df.columns))//numscans, axis=1).mean()
    mean_df.columns = well_list
    sd_df = df.groupby(np.arange(len(df.columns))//numscans, axis=1).std()
    sd_df.columns = well_list
    os.makedirs((os.path.join(graph_directory, folder)), exist_ok=True)
    out_dir = os.path.join(graph_directory, folder)
    
    c = 0
    for column in mean_df,sd_df:
        while c in range(len(well_list)):
            fig1, ax1 = plt.subplots()
            ax1.set_title(well_list[c])
            ax1.set_xlabel("Hour")
            ax1.set_ylabel(ylabel)
            mean_df2 = mean_df[well_list[c]]
            sd_df2 = sd_df[well_list[c]]
            #change the code below, for different graphs
            mean_df2.plot.line(yerr=sd_df2)
            plt.savefig(out_dir + '\\' + well_list[c] + '.png', bbox_inches='tight')
            plt.close()
            c += 1

In [ ]:
plot(df2.loc[:,::5],r'pct_pplt',numscans,well_list,r"Percent Proplatelet-Producing MKs")
plot(df2.loc[:,1::5],r'area_mk',numscans,well_list,r"Total MK Area (mm²)")
plot(df2.loc[:,2::5],r'area_pplt',numscans,well_list,r"Total Pplt Area (mm²)")
plot(df2.loc[:,3::5],r'perimeter_mk',numscans,well_list,r"Total MK Perimeter Area (mm²)")
plot(df2.loc[:,4::5],r'perimeter_pplt',numscans,well_list,r"Total Pplt Perimeter Area (mm²)")

# fluoProcessing
- Currently one pipeline exists "quantifying" ARWs transduced MKs
- **In Progress** - Fully capable of analyzing fluo from the incucyte, but the pipeline is not fleshed out.

In [ ]:
if fluo is not False:
    
    imdir_mk_label = os.path.join(output_directory,"mk_labels");
    mk_list = sorted_nicely(glob.glob(os.path.join(imdir_mk_label,"*.tiff")));
    imdir_pplt_label = os.path.join(output_directory,"proplatelet_labels");
    pplt_list = sorted_nicely(glob.glob(os.path.join(imdir_pplt_label,"*.tiff")));
    label_list = mk_list + pplt_list
    
    if len(mk_list) and len(pplt_list) != len(single_list):
        print("number of fluo single_images doesn't match number of phase single_images")
        break;
    else:
        pass;
    
    if fluo == 1:
        
        fluo_list = single_list + rfp_list + gfp_list + label_list
        fluo_list = sorted_nicely(fluo_list)
        
        with open(os.path.join(image_directory,"fluo_list.txt"), 'w') as f:
            for item in fluo_list:
                f.write("{}\n".format(item))
        
        path_to_fluo_pipeline =  path_to_coloc_pipeline
        
    elif fluo == 2:
        
        fluo_list = single_list + rfp_list + label_list
        fluo_list = sorted_nicely(fluo_list)
        
        with open(os.path.join(image_directory,"fluo_list.txt"), 'w') as f:
            for item in fluo_list:
                f.write("{}\n".format(item))
                
        path_to_fluo_pipeline = path_to_rfp_pipeline
        
    elif fluo == 3:
        
        fluo_list = single_list + gfp_list + label_list
        fluo_list = sorted_nicely(fluo_list)
    
        with open(os.path.join(image_directory,"fluo_list.txt"), 'w') as f:
            for item in fluo_list:
                f.write("{}\n".format(item))
                      
        path_to_fluo_pipeline = path_to_gfp_pipeline
    
    else:
        print("Processing phase skeleton pipeline...")       
        
else:
    pass;

## (2) Fluorescence Pipeline
Use subprocess to run CellProfiler on the phase & 16-bit fluo images to be processed. 3 pipes have been created to measure gfp, rfp, & co-stained MKs/Pplts. Fluo pipes have been placed in if/else statements for instances where fluo is not present -> will proceed to phase skeleton pipe.

In [ ]:
if fluo is not False:
    
    os.makedirs(os.path.join(image_directory, "fluo"), exist_ok=True)
    fluo_directory = os.path.join(image_directory,"fluo");
else:
    pass;

In [ ]:
if fluo is not False:
    
    command = (path_to_cellprofiler,"--run-headless","--pipeline={}".format(path_to_fluo_pipeline),
           "--file-list={}".format(os.path.join(image_directory,"fluo_list.txt")),
           "--image-directory={}".format(image_directory),"--output-directory={}".format(fluo_directory))

    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)

    out, err = process.communicate()
else:
    pass;

## (2) Quantify MK/Proplatelet Fluorescence
From the csvs generated by CellProfiler, the file 'results_Masked[_]MK/Pplt' is parsed. Fluorescent intensities and polarity within MKs are quantified.

In [ ]:
def pct_pos(fluo_obj,total_obj):
    try:
        return ((fluo_obj/(total_obj))*100)
    except ZeroDivisionError:
        pass;

In [ ]:
if fluo is not False:
    
# move raw "results_" files to separate folder
    os.makedirs(os.path.join(fluo_directory, "raw_files"), exist_ok=True)
    dst = os.path.join(fluo_directory, "raw_files")

    for filename in os.listdir(fluo_directory):
        if filename.startswith('results'):
            src = fluo_directory + "\\" + filename
            newdst = dst + '\\' + filename
            move(src, newdst)
            
    if fluo == 1:
        pass;
    
    elif fluo == 2:
        
        result = open(os.path.join(dst,r'results_Image.csv')); 
        df = pd.read_csv(result,index_col = "URL_phase");
        df = df.reindex(index=sorted_nicely(df.index));
        df.drop(df.columns[[5,7,8,9,10,11,12,13,14]], axis=1, inplace=True);
        
        df['Pct_RFP_MK'] = df.apply(lambda row: pct_pos(row['Count_MaskedRedmk'], row['Count_mkobj']), axis=1)
        df['Pct_RFP_Pplt'] = df.apply(lambda row: pct_pos(row['Count_MaskedRedpplt'], row['Count_ppltobj']), axis=1)
        
        df.drop(df.columns[[0,1,2,3,4]], axis=1, inplace=True);
        df = df.set_index("ImageNumber");
        
        df2 = df_format(rfp,rfp_list,df)
        rfp_stack_list = df2[0]
        n = df2[1]
        df2 = df2[2]
        
        csv_format(df2.loc[:,::2],rfp_stack_list,n,fluo_directory,r'PctPositive_RFP_MK.csv');
        csv_format(df2.loc[:,1::2],rfp_stack_list,n,fluo_directory,r'PctPositive_RFP_Pplt.csv');
        
    elif fluo == 3:
        
        result = open(os.path.join(dst,r'results_Image.csv')); 
        df = pd.read_csv(result,index_col = "URL_phase");
        df = df.reindex(index=sorted_nicely(df.index));
        df.drop(df.columns[[5,7,8,9,10,11,12,13,14]], axis=1, inplace=True);
        
        df['Pct_GFP_MK'] = df.apply(lambda row: pct_pos(row['Count_MaskedGreenmk'], row['Count_mkobj']), axis=1)
        df['Pct_GFP_Pplt'] = df.apply(lambda row: pct_pos(row['Count_MaskedGreenpplt'], row['Count_ppltobj']), axis=1)
        
        df.drop(df.columns[[0,1,2,3,4]], axis=1, inplace=True);
        df = df.set_index("ImageNumber");
        
        gfp_single_list = sorted_nicely(glob.glob(os.path.join(gfp_single,"*.tif")))
        df2 = df_format(gfp,gfp_list,df)
        gfp_stack_list = df2[0]
        n = df2[1]
        df2 = df2[2]
        
        csv_format(df2.loc[:,::2],gfp_stack_list,n,fluo_directory,r'PctPositive_GFP_MK.csv');
        csv_format(df2.loc[:,1::2],gfp_stack_list,n,fluo_directory,r'PctPositive_GFP_Pplt.csv');
    
    else:
        pass;
else:
    pass;

## (2) Fluo Graphs

- Graphs are generated (w/ mean & std_dev) automatically, plotting %positive MKs vs Pplts --> Currently using to measure transfection efficiency. Use the formatted csvs for more specific plotting.

In [ ]:
def fluo_plot(p1,p2,folder,numscans,well_list,ylabel): #mkpctpos,ppltpctpos
    
    mean_df1 = p1.groupby(np.arange(len(p1.columns))//numscans, axis=1).mean()
    mean_df1.columns = well_list
    sd_df1 = p1.groupby(np.arange(len(p1.columns))//numscans, axis=1).std()
    sd_df1.columns = well_list

    mean_df2 = p2.groupby(np.arange(len(p2.columns))//numscans, axis=1).mean()
    mean_df2.columns = well_list
    sd_df2 = p2.groupby(np.arange(len(p2.columns))//numscans, axis=1).std()
    sd_df2.columns = well_list

    c = 0
    for column in mean_df1,sd_df1,mean_df2,sd_df2:
        while c in range(len(well_list)):
            fig1, ax1 = plt.subplots()
            ax1.set_title(well_list[c])
            ax1.set_xlabel("Hour")
            ax1.set_ylabel(ylabel)
            mean_df_final = mean_df1[well_list[c]]
            sd_df_final = sd_df1[well_list[c]]
            mean_df_final2 = mean_df2[well_list[c]]
            sd_df_final2 = sd_df2[well_list[c]]
            mean_df_final.plot.line(yerr=sd_df_final,label='MKs')
            mean_df_final2.plot.line(yerr=sd_df_final2,label='Pplts')
            plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
            plt.savefig(flgraph_directory + '\\' + well_list[c] + '.png', bbox_inches='tight')
            plt.close()
            c += 1

In [ ]:
if fluo is not False:
    
    # purely gfp or rfp, not both yet
    os.makedirs(os.path.join(fluo_directory, "graphs"), exist_ok=True)
    flgraph_directory = os.path.join(fluo_directory, "graphs")
    
    if fluo == 1:
        pass;
    
    elif fluo == 2:
        p1 = df2.loc[:,::2]
        p2 = df2.loc[:,1::2]
        fluo_plot(p1,p2,flgraph_directory,numscans,well_list,r"Percent RFP-Positive")
        
    elif fluo == 3:
        p1 = df2.loc[:,::2]
        p2 = df2.loc[:,1::2]
        fluo_plot(p1,p2,flgraph_directory,numscans,well_list,r"Percent GFP-Positive")    
else:
    pass;

# skelProcessing

- If no fluorescent images were detected in the image directory, the phase_skeleton pipeline will be run.

In [17]:
path_to_sk_pipeline = path_to_skel_pipeline_mice

In [18]:
os.makedirs(os.path.join(image_directory, "skeleton"), exist_ok=True)
skeleton_directory = os.path.join(image_directory,"skeleton");

In [19]:
imdir_pplt_label = os.path.join(output_directory,"proplatelet_labels");
pplt_list = sorted_nicely(glob.glob(os.path.join(imdir_pplt_label,"*.tiff")));
skel_list = single_list + pplt_list
with open(os.path.join(image_directory,"skel_list.txt"), 'w') as f:
    for item in skel_list:
        f.write("{}\n".format(item))

## (3) Skeleton Pipeline
Use subprocess to run CellProfiler on the images/labels to be processed.

In [ ]:
command = (path_to_cellprofiler,"--run-headless","--pipeline={}".format(path_to_sk_pipeline),
           "--file-list={}".format(os.path.join(image_directory,"skel_list.txt")),
           "--image-directory={}".format(image_directory),"--output-directory={}".format(skeleton_directory))

process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    
out, err = process.communicate()

## Parse csvs for network analysis 
Proplatelet structures are measured from the following csvs generated by the Skeleton pipeline.

    1 results_MaskedProtrusions
    2 results_prot_seed (pplt seed points)
    3 results_ppltobj

In [ ]:
os.makedirs(os.path.join(skeleton_directory, "raw_files"), exist_ok=True)
dst = os.path.join(skeleton_directory, "raw_files")
for filename in os.listdir(skeleton_directory):
    if filename.startswith('results'):
        src = skeleton_directory + "\\" + filename
        newdst = dst + '\\' + filename
        move(src, newdst)

In [ ]:
def skcsv(dst,csv,header,columns):
    csv = open(os.path.join(dst,csv));
    df = pd.read_csv(csv, usecols = header, index_col = False);
    df.columns = columns;
    return df;

In [ ]:
mp_head = ["Location_Center_X","Location_Center_Y","Parent_ppltobj"]; #MaskedProtrusions; "Parent_protrusions"
mp_columns = ["Masked_Protrusion_X","Masked_Protrusion_Y","Parent_ppltobj"];
mp_df = skcsv(dst,'results_MaskedProtrusions.csv',mp_head,mp_columns);

In [ ]:
ps_head = ['ImageNumber', 'ObjectNumber', 'Metadata_time', 'Location_Center_X', #protrusion_seeds
       'Location_Center_Y',
       'ObjectSkeleton_NumberBranchEnds_MaskedProtrusionsImage',
       'ObjectSkeleton_NumberNonTrunkBranches_MaskedProtrusionsImage',
       'ObjectSkeleton_NumberTrunks_MaskedProtrusionsImage',
       'ObjectSkeleton_TotalObjectSkeletonLength_MaskedProtrusionsImage'];

ps_columns = ['ImageNumber', 'ObjectNumber', 'Timepoint',
              'ProtrusionSeed_X', 'ProtrusionSeed_Y',
              'NumEndpointsBlue', 'NumBranchGreen', 'NumTrunksRed',
              'TotalObjLength'];

ps_df = skcsv(dst,'results_prot_seed.csv',ps_head,ps_columns);

def i(x):
    return (x[5] + x[6] + x[7]);

x = ps_df.apply(i,axis=1);
ps_df['TotalNodes'] = x;
ps_df = ps_df.add(mp_df,axis='columns',fill_value=0);
ps_df.to_csv(os.path.join(skeleton_directory,'Raw_Skel.csv'));

In [ ]:
po_head = ['ImageNumber', 'ObjectNumber', 'Metadata_time',     #ppltobj
       'Children_MaskedProtrusions_Count', 'Location_Center_X',
       'Location_Center_Y', 'Mean_MaskedProtrusions_Location_Center_X',
       'Mean_MaskedProtrusions_Location_Center_Y',
       'Mean_MaskedProtrusions_Number_Object_Number'];

po_columns = ['ImageNumber', 'ObjectNumber', 'Timepoint', 
              'MaskedProtrusionCount', 'PpltObj_X', 'PpltObj_Y', 
              'MeanPpltObj_X', 'MeanPpltObj_Y', 
              'MeanMaskedProtrusionsNumber'];

po_df = skcsv(dst,'results_ppltobj.csv',po_head,po_columns);
po_df.to_csv(os.path.join(skeleton_directory,'PpltObj.csv'));

# NetworkX

This python program is used to construct graphs to model proplatelet structures from the edges & vertices csvs.

- The edge list is a simple data structure that you'll use to create the graph. Each row represents a single edge of the graph with some edge attributes.
- Node lists are usually optional in networkx and other graph libraries when edge lists are provided because the node names are provided in the edge list's first two columns. However, in this case, there are some node attributes that we'd like to add.

In [ ]:
vertices_csv = open(os.path.join(skeleton_directory,r'vertices.csv'));
v_df = pd.read_csv(vertices_csv);
v_df.columns = ['image_number', 'vertex_number','y','x','labels','kind'];#rename i,j to y,x
v_df = v_df[['vertex_number', 'x','y','labels', 'kind','image_number']];#swap y,x to x,y in vertices_csv
v_df.columns = ['Node','x','y','Labels','Kind','ImageNumber'];#rename i,j to y,x
v_df.to_csv(os.path.join(skeleton_directory,'Vert.csv')); #nodelist (optional w/ edgelist, adds more hashable attributes)

In [ ]:
edges_csv = open(os.path.join(skeleton_directory,r'edges.csv'));
header = ['image_number', 'v1', 'v2', 'length'];
e_df = pd.read_csv(edges_csv, usecols = header, index_col = False);
e_df.columns = ['ImageNumber', 'Node_1', 'Node_2', 'Distance'];
e_df = e_df[['Node_1','Node_2','Distance','ImageNumber']];
e_df.to_csv(os.path.join(skeleton_directory,'Edge.csv')); #edgelist

In [ ]:
im=1
obj=1

In [ ]:
def imslc(df,im): #edgelist/nodelist df,imagenum; slice edgelist by imagenumber
    df = df[df["ImageNumber"]==im]
#     del df['ImageNumber']
    return df;

edgelist = imslc(e_df,im)
nodelist = imslc(v_df,im) #Use these lists for all ppltobjs in image

In [ ]:
def labelList(df,im,obj): #df=ps_df,imnum,objnum
    df1 = df[df["ImageNumber"]==im]
    df2 = df1[df1['Parent_ppltobj']==obj]
    return list(df2["ObjectNumber"].unique());

l = labelList(ps_df,im,obj);
a = [int(i) for i in l];
a[:] = [x - 1 for x in a];
nodelist = nodelist[nodelist.Labels.isin(a)];
b = list(nodelist["Node"]);
# b[:] = [x - 1 for x in b]
edgelist = edgelist[edgelist.Node_2.isin(b)]; #Slice the edgelist further by label -> 1 ppltobj per graph

In [ ]:
def skelPlot(edge,node): #edgelist,nodelist
#     g = nx.DiGraph();
    g = nx.Graph();
    
    # Add edges and attributes
    for i, elrow in edge.iterrows():
        g.add_edge(elrow[0], elrow[1], attr_dict=elrow[2:].to_dict())
    
    # Add node attributes
    for i, nlrow in nodelist.iterrows():
        try:
            g.node[nlrow['Node']].update(nlrow[1:].to_dict())
        except KeyError:
            pass
        
    # Define node positions dict for plotting    
    node_positions = {node[0]: (node[1]['x'], -node[1]['y']) for node in g.nodes(data=True)}
    
    color_map = []
    for node in g:
        if g.node[node]['Kind'] == 'E': #Endpoints changed from blue to red
            color_map.append('red')
#         elif g.node[node]['Kind'] == 'B': 
#             color_map.append('green')
        else:
            color_map.append('black') #Make all nodes except endpoints black
                
    plt.figure(figsize=(8, 12))
    nx.draw(g, pos=node_positions,node_size=7, node_color = color_map)#,with_labels = True)
    plt.title("Test", size=15)
    plt.show()
    return g;

In [ ]:
g = skelPlot(edgelist,nodelist); #Are trunk connection distances=3?

In [ ]:
spl=nx.all_pairs_dijkstra_path_length(g) 
for n in spl:
     print(n[1]) 

In [ ]:
sp=networkx.all_pairs_dijkstra_path(g) 
for n in sp:
     print(n[1]) 

In [ ]:
x = input('First Node: ')
y = input('Second Node: ')

In [ ]:
paths = nx.all_shortest_paths(g, x, y)

for path in paths:
    total_length = 0
    for i in range(len(path)-1):
        source, target = path[i], path[i+1]
        edge = g[source][target]
        length = edge['length']
        total_length += length
    print('{}: {}'.format(path, total_length))

In [ ]:
spl=nx.all_pairs_dijkstra_path_length(g)
for n in spl:
     print(n[1])  

In [ ]:
po_csv = open(os.path.join(skeleton_directory,r'PpltObj.csv'));
header = ["ImageNumber",'ObjectNumber', "PpltObj_X", "PpltObj_Y"];
po_df = pandas.read_csv(po_csv, usecols = header, index_col = False);

def centroid(po,i): #ppltobj csv,image #
    x = po[po["ImageNumber"] == i]; #slice ppltobj.csv by ImageNumber
    del x["ImageNumber"];
    x = x.set_index('ObjectNumber').T.to_dict('list')
    return x;

centroid(po_df,6) #x[1] x[1][0] ->dict slicing